In [1]:
!pip install jina

Processing /home/ashwinkumar/.cache/pip/wheels/bc/2e/e6/2ae6c56fff706eb1c20c20c0b558d42904ad004ff7b2530ffd/jina-2.2.5-py3-none-any.whl


In [2]:
!pip install -U jina

Requirement already up-to-date: jina in /home/ashwinkumar/.local/lib/python3.8/site-packages (2.2.5)


In [4]:
from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('/home/ashwinkumar/Downloads/Article_dataset - Sheet1.csv')
df.head()

,article_headline,article_link
0,Build an Automatic Abstractive Text Summarizer...,https://medium.com/codex/build-an-automatic-ab...
1,creating a website to host your python web app...,https://medium.com/@marcosan93/creating-a-webs...
2,system design choosing between aws lambda and ...,https://iamkanikamodi.medium.com/system-design...
3,upgrade level up resource deployment on aws,https://medium.com/slalom-technology/upgrade-l...
4,cloud computing a comparative analysis between...,https://medium.com/@BangBitTech/cloud-computin...


In [26]:
df = df.drop_duplicates().dropna()
df.shape

(550, 2)

In [28]:
for i in range(5):
    print()
    print(f" article heading : {df.iloc[i, 0]} ".center(80, ' '))
    print()
    print(df.iloc[i, -1])
    print()
    print('-' * 80)


 article heading : Build an Automatic Abstractive Text Summarizer in Ten minutes 

https://medium.com/codex/build-an-automatic-abstractive-text-summarizer-in-ten-minutes-f15f07e54bae

--------------------------------------------------------------------------------

    article heading : creating a website to host your python web application    

https://medium.com/@marcosan93/creating-a-website-to-host-your-python-web-application-f06f694a87e8

--------------------------------------------------------------------------------

  article heading : system design choosing between aws lambda and aws fargate   

https://iamkanikamodi.medium.com/system-design-choosing-between-aws-lambda-and-aws-fargate-d586f28da34

--------------------------------------------------------------------------------

         article heading : upgrade level up resource deployment on aws          

https://medium.com/slalom-technology/upgrade-level-up-resource-deployment-on-aws-65c67ac3ab9d

------------------------

In [29]:
docs = DocumentArray()
for i in range(df.shape[0]):
    heading = df.iloc[i, 0]
    link = df.iloc[i, -1]
    doc = Document(text = heading)
    doc.tags['description'] = link
    docs.append(doc)
    
docs

<jina.types.arrays.document.DocumentArray (length=550) at 139986358364144>

In [31]:
docs[0].json()

'{\n  "id": "5e42818a-3ecc-11ec-b9ae-81e31a18cf28",\n  "mime_type": "text/plain",\n  "tags": {\n    "description": "https://medium.com/codex/build-an-automatic-abstractive-text-summarizer-in-ten-minutes-f15f07e54bae"\n  },\n  "text": "Build an Automatic Abstractive Text Summarizer in Ten minutes"\n}'

In [32]:
Document(text ="text")
Document(content ="content")
Document(uri = "path" )

In [33]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name="error_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
        install_requirements = True
    )
    .add(
        name="error_text_indexer",
        uses='jinahub://SimpleIndexer',
        install_requirements = True
    )
)

In [35]:
flow

In [17]:
!pip install torch

In [16]:
!pip install transformers

  Using cached transformers-4.12.3-py3-none-any.whl (3.1 MB)


In [36]:
!rm -rf workspace

with flow:
    flow.index(
      inputs=docs,
        docs = docs,
        parameters = {'name' : 'somethign', 'xyz' : 'asdf'}
  )

⠋ 0/3 waiting error_text_encoder error_text_indexer gateway to be ready...

           Flow@125626[I]:🎉 Flow is ready to use!                                         
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:47109
	🔒 Private network:	192.168.10.218:47109


In [44]:
query = Document(text = input('Query product : '))
with flow:
    response = flow.search(inputs = query, return_results = True)

Query product : lamda
⠋ 0/3 waiting error_text_encoder error_text_indexer gateway to be ready...        

           Flow@125626[I]:🎉 Flow is ready to use!                                         
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:44123
	🔒 Private network:	192.168.10.218:44123


In [45]:
matches = response[0].docs[0].matches
matches

<jina.types.arrays.match.MatchArray (length=20) at 139986361333744>

In [46]:
for m in matches:
    print()
    print(f" article heading : {m.text} ".center(80, ' '))
    print()
    print(m.tags['description'])
    print()
    print('-' * 80)


                        article heading : azure purview                         

https://medium.com/microsoftazure/azure-purview-83817fc50922

--------------------------------------------------------------------------------

                       article heading : azure vocabulary                       

https://medium.com/reaching-for-the-cloud/azure-vocabulary-66e66ccd46b6

--------------------------------------------------------------------------------

                article heading : automating etl with aws lambda                

https://medium.com/@ashwin_muthiah/automating-etl-with-aws-lambda-97b9e3404929

--------------------------------------------------------------------------------

                      article heading : databricks on aws                       

https://medium.com/@prateek.dubey/databricks-on-aws-eef043ae937f

--------------------------------------------------------------------------------

        article heading : getting started with automl and aws 